Table 5: Alcohol-specific age-standardised death rates per 100,000 population, deaths registered in Northern Ireland, 2001 to 2016

In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/causesofdeath/datasets/alcoholspecificdeathsintheukmaindataset')
    sheet = scraper.distribution().as_databaker()

In [2]:
tab = next(tab for tab in sheet if tab.name=='Table 5')

In [3]:
observations = tab.excel_ref('D7').expand(DOWN).expand(RIGHT).is_not_blank()

In [4]:
observations

{<L8 9.7>, <E17 13.9>, <P10 16.1>, <P23 21.6>, <J21 11.0>, <P8 16.9>, <G14 17.5>, <D23 303.0>, <L19 8.4>, <K21 8.9>, <O15 21.2>, <K14 7.7>, <O8 20.5>, <K23 8.2>, <J12 7.2>, <O13 21.5>, <L20 9.4>, <D8 194.0>, <J16 10.1>, <N8 141.0>, <F9 10.3>, <L14 12.1>, <F21 14.5>, <K22 9.5>, <K19 4.8>, <L16 12.6>, <G19 13.9>, <P16 18.9>, <O22 22.2>, <I13 73.0>, <D22 289.0>, <P14 18.5>, <J13 8.9>, <F18 12.8>, <G11 16.1>, <L10 10.4>, <J11 8.0>, <G13 16.9>, <Q19 21.2>, <E20 12.9>, <O18 19.7>, <O16 22.3>, <J9 8.4>, <P17 17.2>, <E8 13.5>, <I8 53.0>, <N11 155.0>, <N16 176.0>, <I16 85.0>, <K18 7.9>, <P18 16.6>, <P19 15.3>, <I21 97.0>, <F7 10.6>, <E21 16.4>, <O12 20.6>, <J20 7.3>, <J8 7.4>, <L18 12.3>, <N12 153.0>, <N14 165.0>, <N9 112.0>, <O20 18.7>, <D18 244.0>, <D14 243.0>, <F19 10.5>, <Q21 25.3>, <I12 57.0>, <G21 18.3>, <E11 14.2>, <P22 18.9>, <Q8 24.1>, <D15 249.0>, <I14 78.0>, <L21 13.5>, <L12 9.3>, <O10 19.3>, <I10 62.0>, <E14 15.6>, <P12 17.3>, <E10 13.5>, <O9 16.2>, <G23 19.7>, <F14 13.6>, <G10 15.4

In [5]:
Sex = tab.excel_ref('D5').expand(RIGHT).is_not_whitespace()
Sex

{<D5 'All Persons'>, <N5 'Males'>, <I5 'Females'>}

In [6]:
deaths = tab.excel_ref('D6').expand(RIGHT).is_not_blank()
deaths

{<I6 'Deaths'>, <O6 'Rate per 100,000 males1'>, <F6 'Lower 95% confidence limit'>, <J6 'Rate per 100,000 females1'>, <K6 'Lower 95% confidence limit'>, <Q6 'Upper 95% confidence limit'>, <D6 'Deaths'>, <N6 'Deaths'>, <G6 'Upper 95% confidence limit'>, <L6 'Upper 95% confidence limit'>, <E6 'Rate per 100,000 people1'>, <P6 'Lower 95% confidence limit'>}

In [7]:
# lc = tab.excel_ref('D6').expand(RIGHT).filter(contains_string('Lower 95% confidence limit'))
# lc

In [8]:
# uc = tab.excel_ref('F6').expand(RIGHT).filter(contains_string('Upper 95% confidence limit'))
# uc

In [9]:
Year = tab.excel_ref('B7').expand(DOWN) - tab.excel_ref('B24').expand(DOWN)  
Year

{<B12 2006.0>, <B17 2011.0>, <B8 2002.0>, <B14 2008.0>, <B18 2012.0>, <B23 2017.0>, <B19 2013.0>, <B22 2016.0>, <B9 2003.0>, <B16 2010.0>, <B21 2015.0>, <B7 2001.0>, <B11 2005.0>, <B20 2014.0>, <B13 2007.0>, <B10 2004.0>, <B15 2009.0>}

In [10]:
Dimensions = [
            HDim(Year,'Year',DIRECTLY,LEFT),
            HDim(Sex,'Sex',CLOSEST,LEFT),
            HDim(deaths, 'Alcohol Specific Deaths',DIRECTLY,ABOVE),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','People')
            ]

In [11]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
if is_interactive():
    savepreviewhtml(c1)

In [12]:
new_table = c1.topandas()
new_table

,OBS,Year,Sex,Alcohol Specific Deaths,Measure Type,Unit
0,178.0,2001.0,All Persons,Deaths,Count,People
1,12.4,2001.0,All Persons,"Rate per 100,000 people1",Count,People
2,10.6,2001.0,All Persons,Lower 95% confidence limit,Count,People
3,14.3,2001.0,All Persons,Upper 95% confidence limit,Count,People
4,61.0,2001.0,Females,Deaths,Count,People
5,8.2,2001.0,Females,"Rate per 100,000 females1",Count,People
6,6.3,2001.0,Females,Lower 95% confidence limit,Count,People
7,10.5,2001.0,Females,Upper 95% confidence limit,Count,People
8,117.0,2001.0,Males,Deaths,Count,People
9,17.0,2001.0,Males,"Rate per 100,000 males1",Count,People


In [13]:
new_table['Year'] = pd.to_numeric(new_table['Year'], errors='coerce').fillna(0)

In [14]:
new_table['Year'] = new_table['Year'].astype(int)

In [15]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [16]:
# new_table['Year'] = new_table['Year'].astype(str)

In [17]:
# new_table['Value'] = new_table['Value'].astype(int)

In [18]:
new_table.dtypes

Value                      float64
Year                         int32
Sex                         object
Alcohol Specific Deaths     object
Measure Type                object
Unit                        object
dtype: object

In [19]:
new_table

,Value,Year,Sex,Alcohol Specific Deaths,Measure Type,Unit
0,178.0,2001,All Persons,Deaths,Count,People
1,12.4,2001,All Persons,"Rate per 100,000 people1",Count,People
2,10.6,2001,All Persons,Lower 95% confidence limit,Count,People
3,14.3,2001,All Persons,Upper 95% confidence limit,Count,People
4,61.0,2001,Females,Deaths,Count,People
5,8.2,2001,Females,"Rate per 100,000 females1",Count,People
6,6.3,2001,Females,Lower 95% confidence limit,Count,People
7,10.5,2001,Females,Upper 95% confidence limit,Count,People
8,117.0,2001,Males,Deaths,Count,People
9,17.0,2001,Males,"Rate per 100,000 males1",Count,People


In [20]:
nt1 = new_table[new_table['Alcohol Specific Deaths'] == 'Lower 95% confidence limit']

In [21]:
nt1

,Value,Year,Sex,Alcohol Specific Deaths,Measure Type,Unit
2,10.6,2001,All Persons,Lower 95% confidence limit,Count,People
6,6.3,2001,Females,Lower 95% confidence limit,Count,People
10,13.9,2001,Males,Lower 95% confidence limit,Count,People
14,11.6,2002,All Persons,Lower 95% confidence limit,Count,People
18,5.5,2002,Females,Lower 95% confidence limit,Count,People
22,16.9,2002,Males,Lower 95% confidence limit,Count,People
26,10.3,2003,All Persons,Lower 95% confidence limit,Count,People
30,6.4,2003,Females,Lower 95% confidence limit,Count,People
34,13.2,2003,Males,Lower 95% confidence limit,Count,People
38,11.7,2004,All Persons,Lower 95% confidence limit,Count,People


In [22]:
nt1.columns = ['Lower 95% confidence limit' if x=='Value' else x for x in new_table.columns]

In [23]:
nt2 = new_table[new_table['Alcohol Specific Deaths'] == 'Upper 95% confidence limit']

In [24]:
nt2

,Value,Year,Sex,Alcohol Specific Deaths,Measure Type,Unit
3,14.3,2001,All Persons,Upper 95% confidence limit,Count,People
7,10.5,2001,Females,Upper 95% confidence limit,Count,People
11,20.1,2001,Males,Upper 95% confidence limit,Count,People
15,15.5,2002,All Persons,Upper 95% confidence limit,Count,People
19,9.7,2002,Females,Upper 95% confidence limit,Count,People
23,24.1,2002,Males,Upper 95% confidence limit,Count,People
27,14.0,2003,All Persons,Upper 95% confidence limit,Count,People
31,10.7,2003,Females,Upper 95% confidence limit,Count,People
35,19.2,2003,Males,Upper 95% confidence limit,Count,People
39,15.4,2004,All Persons,Upper 95% confidence limit,Count,People


In [25]:
nt2.columns = ['Upper 95% confidence limit' if x=='Value' else x for x in new_table.columns]

In [26]:
new_table.count()

Value                      204
Year                       204
Sex                        204
Alcohol Specific Deaths    204
Measure Type               204
Unit                       204
dtype: int64

In [27]:
new_table = new_table[new_table['Alcohol Specific Deaths'] != 'Upper 95% confidence limit']

In [28]:
new_table = new_table[new_table['Alcohol Specific Deaths'] != 'Lower 95% confidence limit']

In [29]:
Final_table = pd.merge(new_table, nt1, how = 'inner', on = ['Year','Sex'])

In [30]:
Final_table.head(2)

,Value,Year,Sex,Alcohol Specific Deaths_x,Measure Type_x,Unit_x,Lower 95% confidence limit,Alcohol Specific Deaths_y,Measure Type_y,Unit_y
0,178.0,2001,All Persons,Deaths,Count,People,10.6,Lower 95% confidence limit,Count,People
1,12.4,2001,All Persons,"Rate per 100,000 people1",Count,People,10.6,Lower 95% confidence limit,Count,People


In [31]:
Final_table = Final_table[['Value','Year','Sex','Alcohol Specific Deaths_x','Measure Type_x','Unit_x',
                           'Lower 95% confidence limit']]

In [32]:
Final_table = pd.merge(Final_table, nt2, how = 'inner', on = ['Year','Sex'])

In [33]:
Final_table.head(1)

,Value,Year,Sex,Alcohol Specific Deaths_x,Measure Type_x,Unit_x,Lower 95% confidence limit,Upper 95% confidence limit,Alcohol Specific Deaths,Measure Type,Unit
0,178.0,2001,All Persons,Deaths,Count,People,10.6,14.3,Upper 95% confidence limit,Count,People


In [34]:
Final_table = Final_table[['Year','Sex','Alcohol Specific Deaths_x','Measure Type','Value','Lower 95% confidence limit', 'Upper 95% confidence limit','Unit']]

In [35]:
Final_table

,Year,Sex,Alcohol Specific Deaths_x,Measure Type,Value,Lower 95% confidence limit,Upper 95% confidence limit,Unit
0,2001,All Persons,Deaths,Count,178.0,10.6,14.3,People
1,2001,All Persons,"Rate per 100,000 people1",Count,12.4,10.6,14.3,People
2,2001,Females,Deaths,Count,61.0,6.3,10.5,People
3,2001,Females,"Rate per 100,000 females1",Count,8.2,6.3,10.5,People
4,2001,Males,Deaths,Count,117.0,13.9,20.1,People
5,2001,Males,"Rate per 100,000 males1",Count,17.0,13.9,20.1,People
6,2002,All Persons,Deaths,Count,194.0,11.6,15.5,People
7,2002,All Persons,"Rate per 100,000 people1",Count,13.5,11.6,15.5,People
8,2002,Females,Deaths,Count,53.0,5.5,9.7,People
9,2002,Females,"Rate per 100,000 females1",Count,7.4,5.5,9.7,People


In [36]:
Final_table.columns = ['Alcohol Specific Deaths' if x=='Alcohol Specific Deaths_x' else x for x in Final_table.columns]

In [37]:
def user_perc(x,y):
    
    if str(x) == 'Deaths':
        return ''
    else:
        return y
    
Final_table['Lower 95% confidence limit'] = Final_table.apply(lambda row: user_perc(row['Alcohol Specific Deaths'],row['Lower 95% confidence limit']), axis = 1)

In [38]:
def user_perc(x,y):
    
    if str(x) == 'Deaths':
        return ''
    else:
        return y
    
Final_table['Upper 95% confidence limit'] = Final_table.apply(lambda row: user_perc(row['Alcohol Specific Deaths'],row['Upper 95% confidence limit']), axis = 1)

In [39]:
Final_table['Alcohol Specific Deaths'] = Final_table['Alcohol Specific Deaths'].str.rstrip('1')

In [40]:
Final_table['Sex'] = Final_table['Sex'].map(
    lambda x: {
        'All Persons' : 'T', 
        'Males' : 'M',
        'Females': 'F'         
        }.get(x, x))

In [41]:
Final_table['Age'] =  'All'

In [42]:
Final_table['Geography'] =  'N92000002'

In [43]:
Final_table

,Year,Sex,Alcohol Specific Deaths,Measure Type,Value,Lower 95% confidence limit,Upper 95% confidence limit,Unit,Age,Geography
0,2001,T,Deaths,Count,178.0,,,People,All,N92000002
1,2001,T,"Rate per 100,000 people",Count,12.4,10.6,14.3,People,All,N92000002
2,2001,F,Deaths,Count,61.0,,,People,All,N92000002
3,2001,F,"Rate per 100,000 females",Count,8.2,6.3,10.5,People,All,N92000002
4,2001,M,Deaths,Count,117.0,,,People,All,N92000002
5,2001,M,"Rate per 100,000 males",Count,17.0,13.9,20.1,People,All,N92000002
6,2002,T,Deaths,Count,194.0,,,People,All,N92000002
7,2002,T,"Rate per 100,000 people",Count,13.5,11.6,15.5,People,All,N92000002
8,2002,F,Deaths,Count,53.0,,,People,All,N92000002
9,2002,F,"Rate per 100,000 females",Count,7.4,5.5,9.7,People,All,N92000002


In [44]:
Final_table.count()

Year                          102
Sex                           102
Alcohol Specific Deaths       102
Measure Type                  102
Value                         102
Lower 95% confidence limit    102
Upper 95% confidence limit    102
Unit                          102
Age                           102
Geography                     102
dtype: int64